In [3]:

from clickhouse_driver import Client
client = Client('bigdataldn-demo-clickhouse-dbeech-demo.aivencloud.com',
                user='avnadmin',
                password='***',
                secure=True,
                port='13223',
                )

client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',),
 ('default',),
 ('information_schema',),
 ('service_bigdataldn-demo-kafka',),
 ('system',)]

In [4]:
first_query = 'select lat, long from default.bus_positions where (veh = 1344) AND (spd > 15.0) limit 500'
out = client.execute(first_query)
len(out)

0

In [6]:
count_query = 'select count() from default.bus_positions'
client.execute(count_query)

[(59862,)]

In [7]:
next_query = 'select lat, long from default.bus_positions where  (spd > 45.0) limit 500'
out = client.execute(next_query)
len(out)

0

In [ ]:
import folium
m = folium.Map(location=[60.40, 25.1067], zoom_start=11)
for point in out:
    try:
        folium.Marker(point).add_to(m)
    except:
        pass
    
m

In [ ]:
marker_colors = ['red', 'darkred',  'lightred', 'orange', 'beige',
                 'green', 'darkgreen', 'lightgreen',
                 'blue', 'darkblue', 'cadetblue', 'lightblue',
                 'purple',  'darkpurple', 'pink',
                 'white', 'gray', 'lightgray', 'black']


                 

In [17]:
out = client.execute('select max (spd) from default.bus_positions')
out


[(24.899999618530273,)]

This is an interesting one as it lets us map colors to speed :)


In [11]:
import math
def colorise(speed_in: float)-> str:
    return marker_colors[math.floor(speed_in/50)]

In [12]:
import folium
def make_map(my_points):
    map = folium.Map(location=[60.40, 25.1067], zoom_start=11)

    for point in my_points:
        try:
            folium.Marker(location=[point[0], point[1]],  icon=folium.Icon(color=colorise(point[2])) ).add_to(map)
        except:
            pass

    return map

In [15]:
import folium
next_query = 'select lat, long, spd from default.bus_positions limit 500'
out = client.execute(next_query)

map = folium.Map(location=[60.40, 25.1067], zoom_start=11)
color_str =''
for point in out:
    try:
        folium.Marker(location=[point[0], point[1]], color = colorise(point[2])).add_to(map)

    except:
        pass

        
map